<a href="https://colab.research.google.com/github/Janemura/CTF-Challenges/blob/main/Entertainmenthub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import sqlite3
from datetime import datetime
import hashlib

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect("entertainment_mis.db")
cursor = conn.cursor()

# Create tables if they do not exist
cursor.execute('''CREATE TABLE IF NOT EXISTS artists (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT,
                    contact TEXT,
                    address TEXT,
                    genre TEXT,
                    date_added TEXT)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS transactions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    artist_name TEXT,
                    action_type TEXT,
                    event_date TEXT,
                    description TEXT,
                    timestamp TEXT)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS payments (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    artist_name TEXT,
                    amount_due REAL,
                    payment_method TEXT,
                    status TEXT,
                    timestamp TEXT)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                    username TEXT PRIMARY KEY,
                    password TEXT,
                    role TEXT)''')
conn.commit()

# Add a default admin user if none exists
def create_default_admin():
    cursor.execute("SELECT * FROM users WHERE username = 'admin'")
    if cursor.fetchone() is None:
        password_hash = hashlib.sha256("admin123".encode()).hexdigest()
        cursor.execute("INSERT INTO users (username, password, role) VALUES (?, ?, ?)", ("admin", password_hash, "admin"))
        conn.commit()

create_default_admin()

# Utility function to display messages
def display_message(message):
    output = widgets.Output()
    with output:
        clear_output()
        print(message)
    display(output)

# User Authentication
def login_form():
    username = widgets.Text(description="Username:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    password = widgets.Password(description="Password:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    login_button = widgets.Button(description="Login", style={'font_size': '16px'})

    def on_login(b):
        cursor.execute("SELECT * FROM users WHERE username = ?", (username.value,))
        user = cursor.fetchone()

        if user and user[1] == hashlib.sha256(password.value.encode()).hexdigest():
            clear_output()
            print(f"Welcome, {user[0]}!")
            if user[2] == "admin":
                run_admin_interface()
            else:
                run_user_interface()
        else:
            display_message("Invalid username or password.")

    login_button.on_click(on_login)
    display(widgets.VBox([username, password, login_button]))

# Form 1: Data Entry Form
def data_entry_form():
    name = widgets.Text(description="Name:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    contact = widgets.Text(description="Contact Info:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    address = widgets.Text(description="Address:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    genre = widgets.Text(description="Genre:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    submit_button = widgets.Button(description="Submit", style={'font_size': '16px'})

    def on_submit(b):
        if name.value and contact.value:
            cursor.execute("INSERT INTO artists (name, contact, address, genre, date_added) VALUES (?, ?, ?, ?, ?)",
                           (name.value, contact.value, address.value, genre.value, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
            conn.commit()
            display_message("Artist registered successfully!")
            name.value = contact.value = address.value = genre.value = ""
        else:
            display_message("Please fill in all required fields.")

    submit_button.on_click(on_submit)
    display(widgets.VBox([name, contact, address, genre, submit_button]))

# Form 2: Transaction/Action Form
def transaction_form():
    artist_name = widgets.Text(description="Artist Name:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    action_type = widgets.Dropdown(options=["Booking", "Performance"], description="Action:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    event_date = widgets.DatePicker(description="Date:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    description = widgets.Text(description="Details:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    submit_button = widgets.Button(description="Submit", style={'font_size': '16px'})

    def on_submit(b):
        if artist_name.value and event_date.value:
            cursor.execute("INSERT INTO transactions (artist_name, action_type, event_date, description, timestamp) VALUES (?, ?, ?, ?, ?)",
                           (artist_name.value, action_type.value, event_date.value.strftime("%Y-%m-%d"), description.value, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
            conn.commit()
            display_message("Transaction recorded successfully!")
            artist_name.value = description.value = ""
        else:
            display_message("Please complete all fields.")

    submit_button.on_click(on_submit)
    display(widgets.VBox([artist_name, action_type, event_date, description, submit_button]))

# Form 3: Update/Modification Form
def update_form():
    artist_name = widgets.Text(description="Artist Name:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    new_contact = widgets.Text(description="New Contact:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    new_address = widgets.Text(description="New Address:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    update_button = widgets.Button(description="Update", style={'font_size': '16px'})

    def on_update(b):
        cursor.execute("SELECT * FROM artists WHERE name = ?", (artist_name.value,))
        artist = cursor.fetchone()
        if artist:
            cursor.execute("UPDATE artists SET contact = ?, address = ? WHERE name = ?",
                           (new_contact.value or artist[2], new_address.value or artist[3], artist_name.value))
            conn.commit()
            display_message("Artist details updated successfully!")
            artist_name.value = new_contact.value = new_address.value = ""
        else:
            display_message("Artist not found.")

    update_button.on_click(on_update)
    display(widgets.VBox([artist_name, new_contact, new_address, update_button]))

# Form 4: Payment or Status Update Form
def payment_form():
    artist_name = widgets.Text(description="Artist Name:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    balance = widgets.FloatText(description="Amount Due:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    payment_method = widgets.Dropdown(options=["Cash", "Credit Card", "PayPal"], description="Method:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    status = widgets.Dropdown(options=["Paid", "Pending"], description="Status:", layout=widgets.Layout(width="450px"), style={'description_width': 'initial', 'font_size': '16px'})
    submit_button = widgets.Button(description="Submit Payment", style={'font_size': '16px'})

    def on_submit(b):
        if artist_name.value:
            cursor.execute("INSERT INTO payments (artist_name, amount_due, payment_method, status, timestamp) VALUES (?, ?, ?, ?, ?)",
                           (artist_name.value, balance.value, payment_method.value, status.value, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
            conn.commit()
            display_message("Payment recorded successfully!")
            artist_name.value = ""
            balance.value = 0.0
        else:
            display_message("Please provide the artist name.")

    submit_button.on_click(on_submit)
    display(widgets.VBox([artist_name, balance, payment_method, status, submit_button]))

# Report Generation with Export Options
def generate_reports():
    overview_button = widgets.Button(description="Overview Report", style={'font_size': '16px'})
    transactions_button = widgets.Button(description="Transactions Report", style={'font_size': '16px'})
    payments_button = widgets.Button(description="Payments Report", style={'font_size': '16px'})

    def show_overview(b):
        df = pd.read_sql_query("SELECT * FROM artists", conn)
        display(df)
        df.to_csv("overview_report.csv", index=False)
        display_message("Overview report exported as 'overview_report.csv'.")

    def show_transactions(b):
        df = pd.read_sql_query("SELECT * FROM transactions", conn)
        display(df)
        df.to_csv("transactions_report.csv", index=False)
        display_message("Transactions report exported as 'transactions_report.csv'.")

    def show_payments(b):
        df = pd.read_sql_query("SELECT * FROM payments", conn)
        display(df)
        df.to_csv("payments_report.csv", index=False)
        display_message("Payments report exported as 'payments_report.csv'.")

    overview_button.on_click(show_overview)
    transactions_button.on_click(show_transactions)
    payments_button.on_click(show_payments)
    display(widgets.VBox([overview_button, transactions_button, payments_button]))

# Admin interface for full access
def run_admin_interface():
    data_entry_form()  # Form 1
    transaction_form()  # Form 2
    update_form()       # Form 3
    payment_form()      # Form 4
    generate_reports()  # Reports with export

# User interface for read-only access
def run_user_interface():
    generate_reports()

# Start the system
login_form()


Welcome, admin!
